<a href="https://colab.research.google.com/github/juliancramos/cnn-pneumonia-covid-detector/blob/main/cnn-pneumonia-covid-detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chest X-Ray Classification: Normal vs. Pneumonia vs. COVID-19

**Authors:** Sergio Ortíz - Julián Ramos - Melissa Ruíz

**Course:** Machine Learning Techniques

**Date:** February 2026

## 1. Executive Summary
Lung diseases such as Pneumonia and COVID-19 are critical health conditions requiring timely diagnosis. Traditional manual interpretation of chest X-rays is subjective and prone to error.

This project aims to develop an automated Deep Learning system using **Convolutional Neural Networks (CNNs)** to classify chest X-ray images into three categories:
1.  **Normal**
2.  **Pneumonia**
3.  **COVID-19**

---
### 1.1 Project Objectives
This project follows a structured workflow designed to meet specific technical requirements:

1.  **Exploratory Data Analysis (EDA):**
    * Explore the dataset to identify key characteristics, including class distribution and image quality.
    * Detect potential biases (e.g., class imbalance) and present visualizations to support design decisions.

2.  **Image Preprocessing:**
    * Implement techniques to improve input quality, such as pixel normalization and resizing.
    * Apply **Data Augmentation** strategies to improve model generalization and prevent overfitting.
    * Justify how these steps prepare the data for the neural network.

3.  **Neural Network Design:**
    * Design a custom **Convolutional Neural Network (CNN)** architecture suitable for classification.
    * Justify the selection of layers (Convolutional, Pooling, Fully Connected), filter sizes, and activation functions.
    * Present the architecture diagram and explain its expected behavior.

4.  **Training and Evaluation:**
    * Train the model using an appropriate optimization strategy and loss function.
    * Evaluate performance on a validation set using robust metrics such as **Precision, Recall, and F1-Score**.
    * Analyze results to identify issues like overfitting and propose technical solutions.